## 감성 분석

### 1-1. 개요
- 나이브 베이즈 분류 모형을 이용하여 문서에 대한 감성 분석(sentiment analysis)
- 감성 분석(sentiment analysis) : 문서에 대해 좋다(positive) 혹은 나쁘다(negative)는 평가를 내리는 것

### 1-2. 목적
- 이 간단한 프로젝트는 텍스트 데이터 평점 및 내용을 사용해서 다양한 인코딩 방법을 통해 각 내용이 영화에 긍정적인 감정과 부정적인 감정을 분석할 수 있는지에 있다.

### 2. 데이터
- 샘플 데이터로 `https://github.com/e9t/nsmc`에 올라와 있는 데이터를 사용
- 번호, 내용, 평점으로 이루져 있으므로 내용을 X, 평점을 y로 저장한다.

### 3. 데이터 전처리

#### 3-1. 데이터 다운로드
- 본인은 해당 명령어 사용이 불가능해서 직접 다운로드 해왔다.

In [ ]:
!wget -nc https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!wget -nc https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

In [2]:
## 유니코드로 인코딩하며 읽기 위해 codecs 패키지를 사용한다.
import codecs
with codecs.open("ratings_train.txt", encoding='utf-8') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:] # header 제외

In [3]:
from pprint import pprint

## 데이터 확인
pprint(data[0])

['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0']


#### 3-2. 독립변수와 종속변수로 분리

In [4]:
X = list(zip(*data))[1]
y = np.array(list(zip(*data))[2], dtype=int)

### 4. CountVectorizer 방법을 사용한 모델 생성
- `CountVectorizer` : 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW 인코딩 벡터를 만든다.
- 다항 나이브 베이즈 모형으로 학습
- Pipeline

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

model1 = Pipeline([
    ('vect', CountVectorizer()),
    ('mb', MultinomialNB()),
])

### 5. 모델 학습

In [6]:
model1.fit(X, y)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('mb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

### 6. 모델 테스트
- 모형의 성능을 보기 위해 테스트 데이터 불러오기.

In [7]:
import codecs
with codecs.open("ratings_test.txt", encoding='utf-8') as f:
    data_test = [line.split('\t') for line in f.read().splitlines()]
    data_test = data_test[1:] # header 제외

#### 6-1. 테스트 데이터 역시 X, y로 분리

In [8]:
X_test = list(zip(*data_test))[1]
y_test = np.array(list(zip(*data_test))[2], dtype=int)

#### 6-2. 결과값 보고서로 출력
- accuracy 0.83

In [9]:
print(classification_report(y_test, model1.predict(X_test)))

             precision    recall  f1-score   support

          0       0.81      0.84      0.83     24827
          1       0.84      0.81      0.82     25173

avg / total       0.83      0.83      0.83     50000



### 7. Tfidf 방법을 사용한 모델 생성
- `TfidfVectorizer` : CountVectorizer와 비슷하지만 TF-IDF 방식으로 단어의 가중치를 조정한 BOW 인코딩 벡터를 만든다.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

model2 = Pipeline([
    ('vect', TfidfVectorizer()),
    ('mb', MultinomialNB()),
])

### 8. 모델 학습

In [11]:
model2.fit(X, y)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...True,
        vocabulary=None)), ('mb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

### 9. 모델 테스트

#### 9-1. 결과 보고서 출력
- accuracy 0.83

In [12]:
print(classification_report(y_test, model2.predict(X_test)))

             precision    recall  f1-score   support

          0       0.81      0.84      0.83     24827
          1       0.84      0.81      0.83     25173

avg / total       0.83      0.83      0.83     50000



### 10. 형태소 분석기를 사용한 모델
- Okt를 사용
- Open Korean Text : 오픈 소스 한국어 분석기. 과거 트위터 형태소 분석기.

In [13]:
from konlpy.tag import Okt
pos_tagger = Okt()

def tokenize_pos(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc)]

C:\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


### 11. 모델 생성

In [14]:
model3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_pos)),
    ('mb', MultinomialNB()),
])

### 12. 모델 학습

In [15]:
%%time
model3.fit(X, y)

Wall time: 7min 43s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...C18>,
        vocabulary=None)), ('mb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

### 13. 테스트 데이터 및 결과 보고서 출력
- accuracy 0.85

In [16]:
print(classification_report(y_test, model3.predict(X_test)))

             precision    recall  f1-score   support

          0       0.85      0.86      0.85     24827
          1       0.86      0.85      0.85     25173

avg / total       0.85      0.85      0.85     50000



### 14. (1,2)-gram 을 사용한 모델 생성
- N그램
    - 단어장 생성에 사용할 토큰의 크기를 결정한다.
    - 모노그램(monogram)은 토큰 하나만 단어로 사용
    - 바이그램(bigram)은 두 개의 연결된 토큰을 하나의 단어로 사용

In [17]:
model4 = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1, 2))),
    ('mb', MultinomialNB()),
])

### 15. 모델 학습

In [18]:
model4.fit(X, y)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
  ...use_idf=True, vocabulary=None)), ('mb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

### 16.  모델 테스트 및 결과 보고서 출력
- accuracy 0.87

In [20]:
print(classification_report(y_test, model4.predict(X_test)))

             precision    recall  f1-score   support

          0       0.86      0.87      0.87     24827
          1       0.87      0.86      0.87     25173

avg / total       0.87      0.87      0.87     50000

